<a href="https://colab.research.google.com/github/mtcarilli/Geuvadis/blob/main/kb_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aligning Geuvadis data with kb-python


This notebook aligns raw bulk RNA FASTQ files from the Geuvadis project [1] with kb-python [2] to produce pseudo-aligned abundances per gene. It aligns samples given a list of Geuvadis run ERR IDs, which can be changed as desired.




<sub><sup> [1] Lappalainen, T., Sammeth, M., Friedländer, M. et al. Transcriptome and genome sequencing uncovers functional variation in humans. Nature 501, 506–511 (2013). https://doi.org/10.1038/nature12531 <sup/><sub/>

<sub><sup>[2] Melsted, P., Booeshaghi, A.S., et al.
Modular, efficient and constant-memory single-cell RNA-seq preprocessing.
Nat Biotechnol  39, 813–818 (2021).
https://doi.org/10.1038/s41587-021-00870-2 <sup/><sub/>

In [ ]:
# install necessary packages
!pip install gget kb-python anndata scanpy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.8 MB/s eta 0:0

In [4]:
# import necessary packages
import scanpy as sc
import anndata as ad
import pandas as pd

In [5]:
# download sample metadata
!wget -O sample_subset_metadata.tsv -L https://raw.githubusercontent.com/mtcarilli/Geuvadis/main/pipeline/sample_subset_metadata.csv

# read in sample metadata
sample_metadata = pd.read_csv('./sample_subset_metadata.tsv')

--2024-02-13 05:39:05--  https://raw.githubusercontent.com/mtcarilli/Geuvadis/main/pipeline/sample_subset_metadata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29044 (28K) [text/plain]
Saving to: ‘sample_subset_metadata.tsv’

sample_subset_metad 100%[===================>]  28.36K  --.-KB/s    in 0.002s  

2024-02-13 05:39:06 (15.7 MB/s) - ‘sample_subset_metadata.tsv’ saved [29044/29044]



In [19]:
# subset to desired ERR IDs (runs), likely based on population
pop = 'Tuscan'
selected_ERRs = sample_metadata[sample_metadata.Ancestry.isin([pop])].Run.values
with open(f'./selected_ERRs.txt', 'w') as f:
    for ERR in selected_ERRs:
        f.write(f"{ERR}\n")

In [ ]:
# create kallisto index and transcript to gene mapping
# !kb ref -i index.idx -g t2g.txt -f1 cdna.fasta $(gget ref --ftp -w dna,gtf human)


# OR download prebuilt reference --> faster
!kb ref -d human -i index.idx -g t2g.txt

[2024-02-12 23:50:57,008]    INFO [download] Skipping download because some files already exist. Use the --overwrite flag to overwrite.


In [7]:
# kb count a given population
# !kb count -i index.idx -g t2g.txt --parity=paired -x bulk -m 9G -t 2 -o ./ERR --tcc ERR188026_1.fastq.gz ERR188026_2.fastq.gz


# !mkfifo R1.gz R2.gz; curl -Ls ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR188/ERR188026/ERR188026_1.fastq.gz R1.gz & curl -Ls ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR188/ERR188026/ERR188026_2.fastq.gz > R2.gz & kb count -i index.idx -g t2g.txt --parity=paired -x bulk -m 9G -t 2 -o ./ERR --tcc R1.gz R2.gz


In [16]:
# loop through all provided Geuvadis ERR runs, align them, store genes.abundance.mtx and genes.abundance.tpm.mtx
%%bash
while IFS= read -r line; do
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR188/${line}/${line}_1.fastq.gz
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR188/${line}/${line}_2.fastq.gz
    kb count -i index.idx -g t2g.txt --parity=paired -x bulk -m 9G -t 2 -o ./out/$line --tcc ${line}_1.fastq.gz ${line}_2.fastq.gz
    rm ${line}_1.fastq.gz
    rm ${line}_2.fastq.gz
    rm -rf ./out/$line/counts_unfiltered/
done < selected_ERRs.txt

Process is interrupted.


In [23]:
# load in first aligned sample
ERR = selected_ERRs[0]
adata_tpm = sc.read_mtx(f'./out/{ERR}/quant_unfiltered/matrix.abundance.gene.tpm.mtx')
adata_tpm.obs.index = [ERR]
adata_tpm.var.index = pd.read_csv(f'./out/{ERR}/quant_unfiltered/genes.txt',sep=' ',header=None).iloc[:,0].values


# loop through all other aligned samples, create concatenated adata object
for ERR in selected_ERRs[1:1]:

  adata_tpm_ = sc.read_mtx(f'./out/{ERR}/quant_unfiltered/matrix.abundance.gene.tpm.mtx')
  adata_tpm_.obs.index = [ERR]
  adata_tpm_.var.index = pd.read_csv(f'./out/{ERR}/quant_unfiltered/genes.txt',sep=' ',header=None).iloc[:,0].values

  adata_tpm = ad.concat((adata_tpm,adata_tpm_),join='outer',fill_value=0.0)

In [30]:
# load in first aligned sample
ERR = selected_ERRs[0]
adata = sc.read_mtx(f'./out/{ERR}/quant_unfiltered/matrix.abundance.gene.mtx')
adata.obs.index = [ERR]
adata.var.index = pd.read_csv(f'./out/{ERR}/quant_unfiltered/genes.txt',sep=' ',header=None).iloc[:,0].values


# loop through all other aligned samples, create concatenated adata object
for ERR in selected_ERRs[1:1]:

  adata_ = sc.read_mtx(f'./out/{ERR}/quant_unfiltered/matrix.abundance.gene.mtx')
  adata_.obs.index = [ERR]
  adata_.var.index = pd.read_csv(f'./out/{ERR}/quant_unfiltered/genes.txt',sep=' ',header=None).iloc[:,0].values

  adata = ad.concat((adata,adata_),join='outer',fill_value=0.0)

matrix([[999999.9]], dtype=float32)

In [ ]:
# save the adata object
adata.write_loom(f'./{pop}.loom')